In [9]:
# This script contains the generator code for producing wireless network layouts and channel losses
# for the work "Spatial Deep Learning for Wireless Scheduling",
# available at https://ieeexplore.ieee.org/document/8664604.

# For any reproduce, further research or development, please kindly cite our JSAC journal paper:
# @Article{spatial_learn,
#    author = "W. Cui and K. Shen and W. Yu",
#    title = "Spatial Deep Learning for Wireless Scheduling",
#    journal = "{\it IEEE J. Sel. Areas Commun.}",
#    year = 2019,
#    volume = 37,
#    issue = 6,
#    pages = "1248-1261",
#    month = "June",
# 
import numpy as np

### Transmitter and Receiver Layout Generation

The `layout_generate` function generates layouts of transmitters and receivers based on certain parameters.

#### Transmitter Layout Generation:

1. **Transmitter Coordinates**:
   - Transmitter coordinates $ (x_{i}, y_{i}) $ are randomly generated within a square field of length $L$: <br><br>
     $ x_i \sim U(0, L) $ <br><br>
     $ y_i \sim U(0, L) $ <br><br>
   - A point in the field represents each transmitter.

#### Receiver Layout Generation:

1. **Number of Receivers**:
   - For each transmitter, a random number of receivers is generated, denoted as $ N_{rx\_i} $, within a predefined range $ [\text{minrx}, \text{maxrx}] $.

2. **Receiver Coordinates**:
   - For each receiver associated with transmitter `i`:
     - A random distance `d` is generated within a predefined range $ [\text{shortest\_directLink\_length}, \text{longest\_directLink\_length}] $.
     - An angle $ \theta $ is randomly chosen within the range $ [0, 2\pi] $.
     - Receiver coordinates $ (x_{\text{rx}\_i}, y_{\text{rx}\_i}) $ are calculated relative to the transmitter location: <br><br>
       $ x_{\text{rx}_i} = x_i + d \cdot \cos(\theta) $ <br><br>
       $ y_{\text{rx}_i} = y_i + d \cdot \sin(\theta) $ <br><br>
     - If the receiver coordinates fall within the field boundary, they are considered valid; otherwise, new coordinates are generated until a valid location is found.

3. **Layout Representation**:
   - The layout of receivers associated with each transmitter `i` is stored as a list of coordinate pairs $ [(x_{\text{rx}_i}, y_{\text{rx}_i})] $.
   - Layouts for all transmitters are stored in a list `layout_rx`.

### Conclusion:
The `layout_generate` function generates layouts of transmitters and receivers within a given field based on random coordinate generation and specified parameters. It ensures that receiver locations are valid and within the field boundaries. This process forms the basis for simulating wireless communication scenarios in various applications.


In [10]:
def layout_generate(general_para):
    N = general_para.n_links
    # first, generate transmitters' coordinates
    tx_xs = np.random.uniform(low=0, high=general_para.field_length, size=[N,1])
    tx_ys = np.random.uniform(low=0, high=general_para.field_length, size=[N,1])
    layout_rx = []
    # generate rx one by one rather than N together to ensure checking validity one by one
    rx_xs = []; rx_ys = []
    tot_links = 0
    n_re = general_para.n_receiver
    for i in range(N):
        n_links = i
        rx_i = []

        num_rx = np.random.randint(general_para.minrx, general_para.maxrx)
        num_rx = min(num_rx,  n_re - tot_links)
        tot_links += num_rx
        for j in range(num_rx): 
            got_valid_rx = False
            while(not got_valid_rx):
                pair_dist = np.random.uniform(low=general_para.shortest_directLink_length, high=general_para.longest_directLink_length)
                pair_angles = np.random.uniform(low=0, high=np.pi*2)
                rx_x = tx_xs[i] + pair_dist * np.cos(pair_angles)
                rx_y = tx_ys[i] + pair_dist * np.sin(pair_angles)
                if(0<=rx_x<=general_para.field_length and 0<=rx_y<=general_para.field_length):
                    got_valid_rx = True
            rx_i.append([rx_x[0], rx_y[0]])
        layout_rx.append(rx_i)
        if(tot_links >= n_re):
            break
    # For now, assuming equal weights and equal power, so not generating them
    layout_tx = np.concatenate((tx_xs, tx_ys), axis=1)
    
    return layout_tx, layout_rx


### Mathematical Equation:
The distance $ d_{ij} $ between transmitter ` i ` and receiver ` j ` can be expressed using summation notation as:

$ d_{ij} = \sqrt{\sum_{k=1}^{N} \left( p_{ik} - q_{jk} \right)^2} $

Where:
- `N` is the number of dimensions (typically 2 for 2D coordinates).
- $ p_{ik} $ and $ q_{jk} $ are the `k`-th dimension coordinates of transmitter `i` and receiver `j` respectively.

### Detailed Explanation:
- The function `distance_generate` calculates distances between transmitters and receivers using the refined summation formula.
- It iterates over each transmitter-receiver pair and computes the Euclidean distance by summing the squares of differences in their coordinates.

### Example:
Let's consider a simple example with two transmitters and two receivers. The coordinates are given as follows:
- Transmitter 1: $ (x_{1}, y_{1}) $
- Transmitter 2: $ (x_{2}, y_{2}) $
- Receiver 1: $ (x_{3}, y_{3}) $
- Receiver 2: $ (x_{4}, y_{4}) $

The distances between each transmitter and receiver are calculated using the refined summation formula as follows:
- $ d_{11} = \sqrt{\left( x_1 - x_3 \right)^2 + \left( y_1 - y_3 \right)^2} $
- $ d_{12} = \sqrt{\left( x_1 - x_4 \right)^2 + \left( y_1 - y_4 \right)^2} $
- $ d_{21} = \sqrt{\left( x_2 - x_3 \right)^2 + \left( y_2 - y_3 \right)^2} $
- $ d_{22} = \sqrt{\left( x_2 - x_4 \right)^2 + \left( y_2 - y_4 \right)^2} $

The refined equation using summation notation provides a concise and mathematically rigorous representation of the distance calculation process. This approach enhances clarity and simplifies understanding, making it easier to interpret and implement the distance calculation algorithm.


In [11]:

def distance_generate(general_para,layout_tx,layout_rx):
    distances = np.zeros((general_para.n_receiver,general_para.n_receiver))
    N = len(layout_rx)
    sum_tx = 0
    for tx_index in range(N):
        num_loops = len(layout_rx[tx_index])
        tx_coor = layout_tx[tx_index]
        for tx_i in range(num_loops):
            sum_rx = 0
            for rx_index in range(N):
                for rx_i in layout_rx[rx_index]:
                    rx_coor = rx_i
                    distances[sum_rx][sum_tx] = np.linalg.norm(tx_coor - rx_coor)
                    sum_rx += 1
            sum_tx += 1
    return distances

The provided Python code generates channel state information (CSI) for a multiple-input multiple-output (MIMO) system based on given parameters.

1. **Initialization**:<br>
   - $ N_{t} $ : Number of transmit antennas.
   - $ L $ : Number of receivers.
   - `distances`: Matrix containing distances between transmitter and receivers.

2. **Shadowing Generation**:
   - $ \text{shadowing}_{i,j,k} $ : Randomly generated shadowing values between transmitter and receiver pairs. This is represented as a matrix of size$ L \times L \times N_t $  with elements sampled from a standard normal distribution.

3. **Large-Scale CSI Calculation**:
   - $ \text{dists}_{i,j,k} $ : Distances between transmitter and receiver pairs. This is represented as a matrix of size$ L \times L \times N_t $  obtained by expanding the `distances` matrix. <br>
   - $ \text{large\_scale\_CSI}_{i,j,k} $ : Large-scale CSI values representing path loss and shadowing effects. Calculated using the formula: <br><br>
     $  \text{large\_scale\_CSI}_{i,j,k} = \dfrac{4.4 \times 10^5}{(\text{dists}_{i,j,k}^{1.88}) \times (10^{\text{shadowing}_{i,j,k} \times \frac{6.3}{20}})} $ <br><br>

   - The large-scale channel gain or large-scale CSI (β) between a transmitter and a receiver in wireless communication systems is often modelled as follows:

    $ \beta = \dfrac{(\text{P}_{\text{tx}} \cdot \text{G}_{\text{tx}} \cdot \text{G}_{\text{rx}} \cdot \lambda^{2})}{((4π)^{2} \cdot \text{d}^{\alpha})} $ <br><br>
    $ \text{P}_{\text{rx},\text{shadow}} = \text{P}_{\text{rx}} \cdot 10^{\frac{X_{shadow}}{10}} $
Where:
- $G_{tx}$ and $G_{rx}$: Antenna gains of the transmitter and receiver, respectively.
- $\text{P}_{\text{tx}}$: Transmit power
- $\text{P}_{\text{rx},\text{shadow}}$: The shadowing effect (Shadowing refers to random variations in the received signal power due to obstacles or environmental conditions. It can be modelled as a log-normal random variable with mean 0 and standard deviation \sigma_{shadow}).
- $X_{shadow}$: Shadowing factor
- `λ`: Wavelength of the carrier frequency.
- `d`: Distance between the transmitter and receiver.
- `α`: path loss exponent, which characterizes the rate of signal attenuation with distance.
- `β`: large-scale channel gain or large-scale CSI  

This equation represents a simplified model of the large-scale channel gain, capturing the basic relationship between distance, antenna gains, carrier frequency, and path loss. It is often used in wireless communication system design and analysis for estimating signal strength and coverage.

The small-scale fading or small-scale channel gain \( h \) is often modelled using the Rayleigh fading model, which assumes that the magnitude of the channel gain follows a Rayleigh distribution. The Rayleigh fading model is commonly represented by the following probability density function (PDF): <br>

  $ f_{h}(h) = \dfrac{h}{\sigma^{2}} \cdot e^{-\frac{h^{2}}{2\sigma^{2}}}$ <br>

Where:
- `h` is the complex channel gain.
- $\sigma^{2}$ is the variance of the channel gain, representing the average received signal power.

This PDF describes the distribution of the magnitude of the small-scale channel gain \( h \), which is characterized by its rapid fluctuations due to multipath propagation. The Rayleigh fading model is widely used in wireless communication to capture the stochastic nature of small-scale fading in various environments.


4. **Small-Scale CSI Calculation**:
   - $ \text{small\_scale\_CSI}_{i,j,k} $ : Small-scale CSI values representing fading effects. Calculated using the formula:<br><br>
     $  \text{small\_scale\_CSI}_{i,j,k} = \dfrac{1}{\sqrt{2}} \times (\text{randn}(L,L,N_t) + j \times \text{randn}(L,L,N_t)) \times \sqrt{\text{large\_scale\_CSI}_{i,j,k}} $ <br><br>
   where:
     - $ \text{randn}(L,L,N_t) $  represents a matrix of size $ L \times L \times N_t $  with elements sampled from a standard normal distribution.
     - $ j $  represents the imaginary unit.

5. **Return**:
   - The function returns the matrix $ \text{small\_scale\_CSI} $  representing the small-scale CSI for the MIMO system.


In [12]:
def CSI_generate(general_para, distances):
    Nt = general_para.N_antennas
    L = general_para.n_receiver
    dists = np.expand_dims(distances,axis=-1)
    shadowing = np.random.randn(L,L,Nt)
    large_scale_CSI = 4.4*10**5/((dists**1.88)*(10**(shadowing*6.3/20)))
    small_scale_CSI = 1/np.sqrt(2)*(np.random.randn(L,L,Nt)+1j*np.random.randn(L,L,Nt))*np.sqrt(large_scale_CSI)
    return small_scale_CSI

The `sample_generate` function generates layouts, distances, and channel state information (CSI) for a given number of layouts based on the provided general parameters.

### Inputs:
- `general_para`: General parameters for layout generation.
- `number_of_layouts`: Number of layouts to generate.
- `norm`: Optional parameter for normalization (default is `None`).

### Outputs:
- `dists`: An array containing the generated distances for each layout.
- `CSIs`: An array containing the generated channel state information for each layout.

### Process:
- For each layout:
  - Generates transmitter and receiver layouts using `layout_generate`.
  - Computes distances between transmitters and receivers using `distance_generate`.
  - Computes channel state information using `CSI_generate`.
  - Collects distances and CSIs for data collection.

### Returns:
- `dists`: Array of distances for all layouts.
- `CSIs`: Array of channel state information for all layouts.

The function organizes the generated data into arrays and returns them for further analysis or processing.


In [13]:
def sample_generate(general_para, number_of_layouts, norm = None):
    print("<<<<<<<<<<<<<{} layouts: {}>>>>>>>>>>>>".format(
        number_of_layouts, general_para.setting_str))
    layouts = []
    dists = []
    CSIs = []
    for i in range(number_of_layouts):
        # generate layouts
        layout_tx, layout_rx = layout_generate(general_para)
        n_re = general_para.n_receiver
        dis = distance_generate(general_para,layout_tx,layout_rx)
        csis = CSI_generate(general_para, dis)
        
        #data collection
        dists.append(dis)
        CSIs.append(csis)
            
    dists = np.array(dists)
    CSIs = np.array(CSIs)
    return dists, CSIs
    